<a href="https://colab.research.google.com/github/119020/NLP_2025_Spring_Materials/blob/main/Tutorial_6_RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RLHF

## Import dependencies

In [ ]:
%pip install transformers trl
!pip install -q -U bitsandbytes
!pip install -q datasets
!pip install unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# del model
# del trainer
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
gc.collect()
!nvidia-smi

Wed Apr 16 19:56:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             68W /  400W |    1251MiB /  40960MiB |     31%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## DPO

In [ ]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from unsloth import FastLanguageModel
from transformers import AutoModelForCausalLM, AutoTokenizer

orgin_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
small_dataset = orgin_dataset.select(range(int(0.005 * len(orgin_dataset))))
print(small_dataset)
print("-"*50)
print(f"question:{small_dataset[0]['chosen'][0]['content']}")
print("-"*50)
print(f"chosen answer:{small_dataset[0]['chosen'][1]['content'][:100]}")
print("-"*50)
print(f"rejected answer:{small_dataset[0]['rejected'][1]['content'][:100]}")
print("-"*50)
print(f"score_chosen:{small_dataset[0]['score_chosen']}")
print("-"*50)
print(f"score_rejected:{small_dataset[0]['score_rejected']}")

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 310
})
--------------------------------------------------
question:Use the pygame library to write a version of the classic game Snake, with a unique twist
--------------------------------------------------
chosen answer:Sure, I'd be happy to help you write a version of the classic game Snake using the pygame library! H
--------------------------------------------------
rejected answer:Sure, here's an example of how to write a version of Snake game with a unique twist using the Pygame
--------------------------------------------------
score_chosen:6.0
--------------------------------------------------
score_rejected:4.0


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
model = FastLanguageModel.get_peft_model(model)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
training_args = DPOConfig(output_dir="Qwen2-0.5B-DPO", logging_steps=10, bf16=True)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=small_dataset)
trainer.train()

Extracting prompt in train dataset (num_proc=12):   0%|          | 0/310 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=12):   0%|          | 0/310 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=12):   0%|          | 0/310 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 310 | Num Epochs = 3 | Total steps = 117
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
10,0.691700,0.010539,0.005732,0.450000,0.004807,-336.992889,-324.710785,-2.770700,-2.771507,0,0,0,0
20,0.688400,-0.007622,-0.020717,0.500000,0.013095,-409.901459,-324.037170,-2.881163,-2.882585,No Log,No Log,No Log,No Log
30,0.682400,-0.037325,-0.072659,0.512500,0.035334,-395.028168,-353.186279,-2.812611,-2.851273,No Log,No Log,No Log,No Log
40,0.648900,0.038233,-0.103269,0.625000,0.141502,-401.356873,-345.175110,-2.736355,-2.734966,No Log,No Log,No Log,No Log
50,0.402000,0.331555,-0.551622,0.912500,0.883177,-416.886322,-366.874451,-2.779846,-2.841599,No Log,No Log,No Log,No Log
60,0.376600,0.298713,-0.703952,0.962500,1.002665,-312.542389,-312.284698,-2.774580,-2.716907,No Log,No Log,No Log,No Log
70,0.329900,0.478842,-0.739059,0.937500,1.217902,-374.312836,-367.830933,-2.799530,-2.838006,No Log,No Log,No Log,No Log
80,0.305900,0.700431,-0.775159,0.962500,1.475590,-474.901276,-369.410217,-2.861488,-2.804312,No Log,No Log,No Log,No Log
90,0.241800,0.757878,-1.071208,0.962500,1.829087,-382.923431,-356.430115,-2.935632,-2.906535,No Log,No Log,No Log,No Log
100,0.168800,0.947241,-1.176052,1.000000,2.123292,-380.014587,-354.372162,-2.733899,-2.695695,No Log,No Log,No Log,No Log


TrainOutput(global_step=117, training_loss=0.42000337645538854, metrics={'train_runtime': 125.0327, 'train_samples_per_second': 7.438, 'train_steps_per_second': 0.936, 'total_flos': 0.0, 'train_loss': 0.42000337645538854, 'epoch': 3.0})

## GRPO

In [ ]:
# del model
# del trainer
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
gc.collect()
!nvidia-smi

Wed Apr 16 20:01:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             65W /  400W |    1739MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer
orgin_dataset = load_dataset("trl-lib/tldr", split="train")
small_dataset = orgin_dataset.select(range(int(0.00005 * len(orgin_dataset))))
print(small_dataset)
print("-"*50)
print(f"question:{small_dataset[0]['prompt'][:100]}")
print("-"*50)
print(f"answer:{small_dataset[0]['completion'][:100]}")

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 5
})
--------------------------------------------------
question:SUBREDDIT: r/relationships

TITLE: I (f/22) have to figure out if I want to still know these girls o
--------------------------------------------------
answer: I still have contact with an old ex's friends but can't stand to see or talk to him. His friends ar


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
model = FastLanguageModel.get_peft_model(model)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Define the reward function, which rewards completions that are close to 20 characters
def reward_len(completions, **kwargs):
    return [-abs(20 - len(completion)) for completion in completions]

In [ ]:
training_args = GRPOConfig(output_dir="Qwen2-0.5B-GRPO", logging_steps=10)
trainer = GRPOTrainer(model=model,reward_funcs=reward_len,args=training_args,train_dataset=small_dataset)
trainer.train()

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 4 to the `num_generations` of 8


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 3 | Total steps = 6
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_len


TrainOutput(global_step=6, training_loss=1.6401211420694988e-05, metrics={'train_runtime': 156.2536, 'train_samples_per_second': 0.096, 'train_steps_per_second': 0.038, 'total_flos': 0.0, 'train_loss': 1.6401211420694988e-05})